In [1]:
import numpy as np
import re
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from pickle import dump
from pickle import load 
from keras.models import load_model
import pickle

Using TensorFlow backend.


In [3]:
# Note: Colab pipeline meant for continuing model training 
# We load the processed character text instead of the ready-to-run X and y data
# because uploading X and y data usually too large to upload. 
# We repeat the preprocessing steps 
with open("process_spenser.txt", "rb") as fp: 
    char_text2 = pickle.load(fp) 
with open("process_shakespeare_final.txt", "rb") as fp: 
    char_text = pickle.load(fp)

In [4]:
# Load mapping
with open("mapping.pkl", "rb") as fp: 
    mapping = pickle.load(fp)
vocab_size = len(mapping)

In [5]:
length = 40 + 1
sequences = []
for i in range(length, len(char_text)): 
    seq = char_text[i-length:i]
    seq_string = ''.join(seq)
    sequences.append(seq_string)
print(len(sequences))

91786


In [6]:
length2 = 40 + 1
sequences2 = []
for i in range(length2, len(char_text2)): 
    seq = char_text2[i-length2:i]
    seq_string = ''.join(seq)
    sequences2.append(seq_string)
print(len(sequences2))

53711


In [7]:
num_char = []
for line in sequences: 
    encoded = [mapping[char] for char in line]
    num_char.append(encoded)
print(len(num_char[0]))

41


In [8]:
num_char2 = []
for line in sequences2: 
    encoded = [mapping[char] for char in line]
    num_char2.append(encoded)
print(len(num_char2[0]))

41


In [9]:
num_char = np.array(num_char)
X, y = num_char[:, :-1], num_char[:, -1]
print(X.shape)
print(y.shape)

(91786, 40)
(91786,)


In [10]:
num_char2 = np.array(num_char2)
X2, y2 = num_char2[:, :-1], num_char2[:, -1]
print(X2.shape)
print(y2.shape)

(53711, 40)
(53711,)


In [11]:
X = np.array([to_categorical(x, num_classes=len(mapping)) for x in X])
y = to_categorical(y, num_classes=len(mapping))
print(X.shape)
print(y.shape)

(91786, 40, 60)
(91786, 60)


In [12]:
X2 = np.array([to_categorical(x, num_classes=len(mapping)) for x in X2])
y2 = to_categorical(y2, num_classes=len(mapping))
print(X2.shape)
print(y2.shape)

(53711, 40, 60)
(53711, 60)


In [ ]:
# For running locally, you can load directly and then stack 
with open("X_training_shakespeare_final.h5", "rb") as fp: 
    X = pickle.load(fp)
with open("y_training_shakespeare_final.h5", "rb") as fp: 
    y = pickle.load(fp)
with open("X_training_spenser.h5", "rb") as fp: 
    X2 = pickle.load(fp)
with open("y_training_spenser.h5", "rb") as fp: 
    y2 = pickle.load(fp)

In [13]:
# Now X_train y_train ready for training
X_train = np.vstack((X2, X))
print(X_train.shape)
y_train = np.vstack((y2, y))
print(y_train.shape)

(145497, 40, 60)
(145497, 60)


In [ ]:
# Load from latest version of model 4 to continue training
model = load_model("model4_3.h5")
print(model.summary())

In [16]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs = 20, batch_size = 64, verbose=1)

Epoch 1/1
91786/91786 [==============================] - 107s 1ms/step - loss: 1.3200 - acc: 0.6026


In [ ]:
# Save the trained model
filename = "model4_4.h5"
model.save(filename)

In [ ]:
# Download from colab to local disk
from google.colab import files
files.download(filename)